In [40]:
%run functions.py

In [70]:
table = 'contract-explorer-233919.ethparis.functions3'
limit = 'LIMIT 20000'

df = load_from_bigquery(table, limit)

In [71]:
target = df.hash
trees = df.tree
# trees = [json.loads(t) for t in trees]
pp = Preprocess().fit_transform(trees)

X_train, X_test, y_train, y_test = train_test_split(pp, target, test_size=0.33,random_state=42)

In [72]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=1000)
toArray = ToArray()
pca = PCA(n_components=10)

pipe = Pipeline([('vectorizer', vectorizer ),
                ('toArray', toArray),
                 ('pca', pca)])


In [73]:
##run pipeline for X_train
result = pipe.fit_transform(X_train)
###run pipeline for X_test
result_test = pipe.transform(X_test)

In [74]:
rf = RandomForestClassifier(n_estimators=48,n_jobs=-1).fit(result,y_train)

In [75]:
rf.score(result_test, y_test)

0.5024242424242424

In [61]:
#dumb model - predict most common class
# target.value_counts()[0]/len(target)

In [62]:
#number of unique function hashes
# len(y_train.unique())

In [63]:
# n=3
# probs = rf.predict_proba(result_test)

# indices = np.argsort(probs, axis=1)[:,-n:]

# top_n_classes = rf.classes_[indices]

# p = copy(probs)
# p.sort(axis=1)
# top_n_probs = p[:,-n:]

In [64]:
# top_n_classes[0:10][:,::-1]

In [65]:
# top_n_probs[0:10][:,::-1]

In [66]:
# y_test[0:10]

In [67]:
from sklearn.linear_model import LogisticRegression

In [68]:
lr = LogisticRegression().fit(result,y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [69]:
lr.score(result_test, y_test)

0.12878787878787878

In [76]:
#pickle
pickle.dump(rf, open( "pickle/rf.p", "wb" ) )
pickle.dump(pipe, open( "pickle/pipe.p", "wb" ) )

In [52]:
pickle.dump(lr, open( "pickle/lr.p", "wb" ) )